
This document contains a mathematical Mixed Integer Linear Programming (MILP) model and its corresponding code. Here's an overview of the report:

- The report is presented as a Jupyter notebook, with each code block of code is explained by markdown cells that explain its function. 
- The formulations for the model are presented here, and following each formulation, the corresponding code is provided below it.
- Approximately 10-12 hours were invested in this project.
- Despite the effort, a feasible solution was not produced. The computation of decision variables takes about 20 minutes and then the - - - process crashes. Here are some strategies attempted to resolve this issue:
- - The Pulp solver, a free tool, is currently in use. One alternative could be to test the model with CPLEX or another commercial solver to see if that enhances performance. The decision variable X is programmed in different ways applicable to use CPLEX for demonstration.
- - Another approach is to refine the input data filtering process to capture only necessary information instead of the entire dataset.

I appologize for not being able to dedicate more time due to my existing workload.






# Input Parameters:

Defined and used per as needed through the document

# Decision Variables:

<b>x<sub>lat</sub></b>: Binary decision variable it's 1 if aircraft <i>a</i> is assigned to fly leg <i>l</i> at time <i>t</i>, and 0 otherwise. 

<b>y<sub>dat</sub></b>: Binary decision variable. It takes a value of 1 if demand <i>d</i> is fulfilled by aircraft <i>a</i> at time <i>t</i>, and 0 otherwise. 

<b>z<sub>la</sub></b>: Binary decision variable. It takes a value of 1 if leg <i>l</i> is flown empty by aircraft <i>a</i>, and 0 otherwise. 

<b>empty_leg_vars<sub>alt</sub></b>: Binary decision variable. It takes a value of 1 if aircraft <i>a</i> flies leg <i>l</i> empty (without passengers) at time <i>t</i>, and 0 otherwise.


## Import libraries

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus
import pandas as pd
# import cplex


## load data

In [ ]:
aircraft_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Aircraft.csv")
airports_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Airports.csv", encoding='ISO-8859-1')
demands_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Demands.csv")
distance_time_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Distance and Flying Time.csv")

Check for missing data

In [ ]:
aircraft_data.isnull().sum()
airports_data.isnull().sum()
demands_file.isnull().sum()
distance_time_file.isnull().sum()


# Scenario 1D: Focus on a single day: 24th July

In [ ]:
specific_date = pd.Timestamp('2022-07-24')
demands_file['ScheduledDepDatetime'] = pd.to_datetime(demands_file['ScheduledDepDatetime'])
demands_file = demands_file[demands_file['ScheduledDepDatetime'].dt.date == specific_date.date()]

# Scenario 2D: Focus on a single day: 24th and 25th July

In [ ]:
# specific_date = [pd.Timestamp('2022-07-24'), pd.Timestamp('2022-07-25')]
# demands_file['ScheduledDepDatetime'] = pd.to_datetime(demands_file['ScheduledDepDatetime'])
# demands_file = demands_file[demands_file['ScheduledDepDatetime'].dt.date.isin([date.date() for date in specific_date])]


## Preprocess

In [ ]:
aircraft_set = aircraft_data['AircraftID'].unique()
demands_set = demands_file['DemandID'].unique()
airports_set = airports_data['Airport Name'].unique() 
L = list(set(zip(distance_time_file['DepAirport'], distance_time_file['ArrAirport'])))
# time_periods = [date.date() for date in specific_date] # for 2 day scenarios
time_periods = [specific_date.date()] # for 1 day scenarios


In [ ]:
print(len(demands_set))
print(len(aircraft_set))
print(len(L))

-- L is problematic because is large so lets filter it

In [ ]:
max_distance = 1500  # 1,500 nautical miles
filtered_distance_time_file = distance_time_file[distance_time_file['Distance'] <= max_distance]
L = list(set(zip(filtered_distance_time_file['DepAirport'], filtered_distance_time_file['ArrAirport'])))

In [ ]:
print(len(L))

## Parameters

In [ ]:
operating_cost_per_hour = 1500  # USD 1,500 per flying hour

operating_costs = {row['DepAirport'] + row['ArrAirport']: row['FlyingTime'] * operating_cost_per_hour
                   for _, row in distance_time_file.iterrows()}

revenue_per_hour = 5000  # USD 5,000 per flying hour

revenue_per_demand = {row['DemandID']: row['EstimatedBlocktime'] * revenue_per_hour
                      for _, row in demands_file.iterrows()}


In [ ]:
# Create a dictionary mapping each flight leg to its estimated block time (if demand exists)
demand_block_time = {(row['DepAirport'], row['ArrAirport']): row['EstimatedBlocktime']
                     for _, row in demands_file.iterrows() if not pd.isna(row['EstimatedBlocktime'])}

# Create a dictionary mapping each flight leg to its flying time
flying_time = {(row['DepAirport'], row['ArrAirport']): row['FlyingTime']
               for _, row in distance_time_file.iterrows()}



In [ ]:
for l in L:
    dep_airport, arr_airport = l

    if l in demand_block_time:
        travel_time = demand_block_time[l]
    else:
        # Flight leg is empty, use travel time values from input data tables
        travel_time = flying_time.get(l, 0)  # Default to 0 if not found

         # Check if the travel time exceeds the maximum flight range
        if travel_time > 4.5:
            # Flight leg requires refueling, add one hour to the travel time
            travel_time += 1

    # operating_costs[l] = travel_time # here is missing the operating cost for each flight leg
    operating_costs[l] = travel_time * operating_cost_per_hour


## Model initialization

In [ ]:
model = LpProblem("Aircraft_Route_Optimization", LpMaximize)
# model = cplex.Cplex()

## Decision Variables

In [ ]:
y_vars = LpVariable.dicts("y", [(d, a, t) for d in demands_set 
                                for a in aircraft_set for t in time_periods], cat=LpBinary)

In [ ]:
x_vars = LpVariable.dicts("x", 
                          [(a, l, t) for a in aircraft_set for l in L for t in time_periods], 
                          cat=LpBinary)

In [ ]:
# # demonstration for testing with CPLEX
# x_var_names = ["x_" + "_".join([str(a), str(l), str(t)]) for a in aircraft_set for l in L for t in time_periods]

# # Add the binary variables to the CPLEX model
# model.variables.add(names=x_var_names, types=[model.variables.type.binary] * len(x_var_names))

# # Now 'x_vars' in the CPLEX model corresponds to the binary decision variables

In [ ]:
#TODO: this coulbe deleted
# z_vars = LpVariable.dicts("z", 
#                           [(a, l) for a in aircraft_set for l in L], 
#                           cat=LpBinary)

In [ ]:
empty_leg_vars = LpVariable.dicts("EmptyLeg", 
                                  [(a, l, t) for a in aircraft_set for l in L for t in time_periods], 
                                  cat=LpBinary)

# Constraints

1. Demand Fulfillment: Each demand must be fulfilled at least once in the planning period.


$$\sum_{t} y_{dt} \geq 1 \quad \text{for all } d$$


In [ ]:
# for d in demands_set:
#     model += lpSum(y_vars[d, a, t] for a in aircraft_set for t in time_periods) >= 1, f"Demand_Fulfillment_{d}"


---above constraint is redundant because it is defined again later. also in formulation, it is missing a and its sumation sign

 
2. Aircraft Route Continuity: Ensures that for each aircraft, the number of arrivals at an airport equals the number of departures.

In [ ]:
#TODO: the formulation representation could be better

$$\sum_{l, S_{la} = k} X_{lat} = \sum_{l, E_{la} = k} X_{lat} \quad \text{for all } a, t, \text{ and } k \text{ in airports}$$


it iterates through flight legs and check number of arrivals for each airport must be equal to number of departures

In [ ]:
for a in aircraft_set:
    for k in airports_set:  
        for t in time_periods:
            arrivals = lpSum(x_vars[a, (dep, arr), t] for (dep, arr) in L if arr == k)
            departures = lpSum(x_vars[a, (dep, arr), t] for (dep, arr) in L if dep == k)
            model += (arrivals == departures), f"Aircraft_Route_Continuity_{a}_{k}_{t}"


3. Aircraft Utilization Limit: Each aircraft cannot exceed its maximum hours of service. 

$$\sum_{l} \sum_{t} x_{lat} \leq \text{HOS}_a \quad \text{for all } a$$


In [21]:
# for a in aircraft_set:
#     for t in time_periods:
#         total_hours_of_operation = lpSum(x_vars[a, l, t] for l in L)
#         model += (total_hours_of_operation <= 12.5), f"Utilization_Limit_{a}_{t}" #did not defined new parameter for HOS because it was 12.5 for all aircrafts

### TODO:correction ###
for a in aircraft_set:
    # Sum across all legs and all time periods for aircraft 'a'
    total_hours_of_operation = lpSum(x_vars[a, l, t] for l in L for t in time_periods)
    model += (total_hours_of_operation <= 12.5), f"Utilization_Limit_{a}"


4. 	Mandatory Rest: After reaching the maximum HOS, the aircraft must observe a mandatory rest period.

$$
\begin{aligned}
&\text{If } \quad l \in L, \quad \sum x_{lat} = \text{HOS}_a, \text{ then} \\
&\quad l \in L, \quad \sum x_{la(t+1)} = 0
\end{aligned}
$$


Given the complexity of directly implementing this as a linear constraint, a practical approach might be to set a utilization limit slightly less than the maximum HOS for each day, thereby implicitly allowing for rest time. This approach simplifies the model while achieving the intended outcome of ensuring rest periods.

 ----- this constraint is written wrong in formulation ----

In [ ]:
#TODO: this constraint is wrong
for a in aircraft_set:
    for t in range(len(time_periods) - 1): 
        current_time = time_periods[t]
        next_time = time_periods[t + 1]
        time_difference = (next_time - current_time).total_seconds() / 3600

        if time_difference >= 12.5:
            # model += (time_difference >= 22.5), f"Mandatory_Rest_{a}_{current_time}"
            time_difference += 10


5.	Flight Leg Assignment: A flight leg can only be assigned if it is either loaded or flown empty.

$$x_{lat} \leq M \cdot z_{la} \quad \text{for all } l, a, \text{ and } t$$


In [ ]:
M = 10000  
for a in aircraft_set:
    for l in L:
        for t in time_periods:
            #model += x_vars[a, l, t] <= M * z_vars[a, l], f"Flight_Leg_Assignment_{a}_{l}_{t}"
            model += x_vars[a, l, t] <= M * empty_leg_vars[a, l, t], f"Flight_Leg_Assignment_{a}_{l}_{t}"


--- this is wrong --- 
it should be:

$$
\forall l \in L, \forall a \in \text{Aircraft Set}, \forall t \in \text{Time Periods}: x_{lat} + y_{lat} \leq 1
$$

In [ ]:
for l in L:
    for a in aircraft_set:
        for t in time_periods:
            model += x_vars[a, l, t] + empty_leg_vars[a, l, t] <= 1, f"Flight_Leg_Assignment_{l}_{a}_{t}"


6. Ensures that each flight leg l is assigned to at most one aircraft 

$$
\sum_{a \in A} \sum_{t \in T} x_{alt} \leq 1 \quad \forall l \in L
$$

In [ ]:
for l in L:
    model += lpSum(x_vars[a, l, t] for a in aircraft_set for t in time_periods) <= 1, f"Assignment_{l}"

---next constraint is not needed. there is no info regarding maxpax----

7. Ensures the passenger capacity of an aircraft is not exceeded for each aircraft and each time period

$$
\sum_{d \in D : (d,a,t) \in y\_vars} RevenuePerDemand_d \cdot y_{dat} \leq MaxPax_a \quad \forall a \in A, \forall t \in T
$$

In [ ]:
for a in aircraft_set:
    for t in time_periods:
        max_capacity = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'MaxPax'].iloc[0]
        assigned_passengers = lpSum(revenue_per_demand[d] * y_vars[d,a,t] for d in demands_set if (d,a,t) in y_vars)
        model += assigned_passengers <= max_capacity, f"Passenger_Capacity_{a}_{t}"


---in next constraint, turnaround time is wrong. it is given az one hour

8.  Ensures an aircraft is available for its next flight only after accounting for the required turn-around time

$$
\sum_{l \in L} \sum_{t_{prev} \in T : t_{prev} < t} x_{al_{t_{prev}}} + TurnAroundTime_a \leq \sum_{l \in L} x_{al_{t+1}} \quad \forall a \in A, \forall t \in T \setminus \{last\}
$$

In [ ]:
for a in aircraft_set:
    for t in time_periods[:-1]: 
        turn_around_time = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'TurnAroundTime'].iloc[0]
        #turn_around_time = 1
        available_time = lpSum(x_vars[a, l, t_prev] for l in L for t_prev in time_periods if t_prev < t) + turn_around_time
        model += available_time <= lpSum(x_vars[a, l, t] for l in L) # should be t+1


9. Ensures that each aircraft is limited to flying no more than one route

$$
\sum_{l \in L} \sum_{t \in T} x_{alt} \leq 1 \quad \forall a \in A
$$

In [ ]:
for a in aircraft_set:
    model += lpSum(x_vars[a, l, t] for l in L for t in time_periods) <= 1, f"Single_Route_Constraint_{a}"


10. Ensure that each demand is assigned to at most one aircraft

$$
\sum_{a \in A} \sum_{t \in T} y_{dat} \leq 1 \quad \forall d \in D

$$

In [ ]:
for d in demands_set:
    model += lpSum(y_vars[d, a, t] for a in aircraft_set for t in time_periods) <= 1, f"Single_Aircraft_Assignment_{d}"

11. Ensure that all demands are satisfied

$$
\sum_{a \in A} \sum_{t \in T} y_{dat} \geq 1 \quad \forall d \in D

$$

In [ ]:
for d in demands_set:
    model += lpSum(y_vars[d, a, t] for a in aircraft_set for t in time_periods) >= 1, f"Satisfy_Demand_{d}"


---next constraint is wrong---

first it should be greater than. second, it ensures that if demand is fulfiled then that leg must be assigned

12. Ensures that aircraft departure time aligns with scheduled departure datetime of demands

$$
x_{adt} \leq y_{dat} \quad \forall d \in D, \forall a \in A, \forall t \in T

$$

In [ ]:
for d in demands_set:
    for a in aircraft_set:
        for t in time_periods:
            model += x_vars[a, d, t] <= y_vars[d, a, t], f"Departure_Sync_{a}_{d}_{t}"


13. Each aircraft starts from its initial location

$$
\sum_{dest \in \text{airports} : (init(a), dest) \in L} x_{a(init(a), dest)t_0} = 1 \quad \forall a \in A

$$

In [ ]:
for a in aircraft_set:
    initial_location = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialLoca'].iloc[0]
    model += lpSum(x_vars[a, (initial_location, dest), time_periods[0]] 
                   for dest in airports_set if (initial_location, dest) in L) == 1, f"Start_From_Initial_Location_{a}"


14. Ensure that the total scheduled hours do not exceed the Initial HOS for each aircraft

$$
\sum_{(dep, arr) \in L} \sum_{t \in T} OperatingCost_{(dep, arr)} \cdot x_{a(dep, arr)t} \leq InitialHOS_a \quad \forall a \in A

$$

In [ ]:
for a in aircraft_set:
    initial_hos = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialHOS'].iloc[0]
    model += lpSum(operating_costs[(dep, arr)] * x_vars[a, (dep, arr), t] 
                   for (dep, arr) in L for t in time_periods) <= initial_hos, f"Max_HOS_{a}"


--- the formulation is wrong. it should be


$$
\sum_{l \in L} \sum_{t \in T} x_{alt} \cdot \text{Flyingtime}_{l} \leq \text{InitialHOS}_{a} \quad \forall a \in A
$$

In [ ]:
for a in aircraft_set:
    initial_hos = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialHOS'].iloc[0]
    #since is 12.5 for all aircrafts, we can use the same parameter
    #initial_hos = 12.5
    total_flight_time = lpSum(x_vars[a, l, t] * flying_time[l] for l in L for t in time_periods)
    model += total_flight_time <= initial_hos, f"Initial_HOS_Constraint_{a}"


16. This constraint defines the conditions under which a flight leg is considered empty. If the aircraft is flying the leg but not serving any demands, this difference will be positive, and the empty_leg_vars will be forced to 1, indicating an empty leg.

$$
empty\_leg_{alt} \geq x_{alt} - \sum_{d \in D : d = l} y_{dat} \quad \forall a \in A, \forall l \in L, \forall t \in T
$$


In [ ]:
for a in aircraft_set:
    for l in L:
        for t in time_periods:
            model += empty_leg_vars[a, l, t] >= x_vars[a, l, t] - lpSum(y_vars[d, a, t] for d in demands_set if d == l), f"Empty_Leg_Definition_{a}_{l}_{t}"

## objective function


Maximize total profit, which is the revenue from fulfilling demands minus the operating costs:


$$
\text{Maximize } Z =  \sum_{d \in D} \sum_{a \in A} \sum_{t \in T} RevenuePerDemand_d \cdot y_{dat} - W_2 * \sum_{l \in L} \sum_{a \in A} \sum_{t \in T} operating_costs_l \cdot x_{lat} - W_3 * \sum_{l \in L} \sum_{a \in A} empty_leg_penalty \cdot z_{la}
$$


Based on 3 cases for each scenarios, Ws can be modified to meet the requirement.

a) Focus of optimization search on maximization of total profit: in this case just make all Ws equal(=1) to focus on profit

b) Focus of optimization search on minimization of empty flying movements: increase the W3 to make sure it is avoiding he empty fliying movement to extend possible

c) Focus of optimization search on minimization of aircraft days used to satisfy the given demands: increasing the W2 to reduce the operational cost by having less days.


In [ ]:
revenue = lpSum(revenue_per_demand[d] * y_vars[d, a, t] for d in demands_set for a in aircraft_set for t in time_periods)

operational_cost = lpSum(operating_costs[l] * x_vars[a, l, t] for a in aircraft_set for l in L for t in time_periods)

empty_leg_penalty = 100000  
empty_leg_cost = lpSum(empty_leg_penalty * empty_leg_vars[a, l, t] for a in aircraft_set for l in L for t in time_periods)

model += revenue - operational_cost - empty_leg_cost, "Total_Profit"


## solve the model

Now for each scenario, Ws must be modified before running the model

In [ ]:
model.solve()

if LpStatus[model.status] == 'Optimal':
    print("Optimal Solution Found!")
    for var in model.variables():
        if var.varValue is not None and var.varValue > 0:
            print(var.name, "=", var.varValue)
else:
    print("No optimal solution found. Status:", LpStatus[model.status])

# model.writeLP("model.lp")

## output

After generationg the feasible solution and verify the validity of the solution, then next step is to format the output to meet the output schema.